In [ ]:
!pip install swarmauri

In [13]:
# Install necessary packages
!pip install gradio --quiet

from swarmauri.vector_stores.concrete.TfidfVectorStore import TfidfVectorStore
from swarmauri.documents.concrete.Document import Document
from swarmauri.conversations.concrete.MaxSystemContextConversation import MaxSystemContextConversation
from swarmauri.messages.concrete.SystemMessage import SystemMessage
from swarmauri.messages.concrete.HumanMessage import HumanMessage
from swarmauri.llms.concrete.GroqModel import GroqModel as LLM
from swarmauri.agents.concrete.RagAgent import RagAgent
import gradio as gr
import os

# Initialize the vector store and add sample documents
vector_store = TfidfVectorStore()
documents = [
    Document(content="Python is a versatile programming language."),
    Document(content="Data science uses machine learning and statistics."),
    Document(content="Python is popular in data science."),
    Document(content="AI advancements are driven by machine learning."),
]
vector_store.add_documents(documents)

# Initialize LLM
os.environ['GROQ_API_KEY'] = 'YOUR_API_KEY'  #replace with api key

API_KEY = os.getenv("GROQ_API_KEY")
if API_KEY:
    llm = LLM(api_key=API_KEY)
else:
    raise ValueError("API Key not found!")

# Filter allowed models
failing_llms = [
    "llama3-70b-8192",
    "llama-3.2-90b-text-preview",
    "mixtral-8x7b-32768",
    "llava-v1.5-7b-4096-preview",
    "llama-guard-3-8b",
]
allowed_models = [model for model in llm.allowed_models if model not in failing_llms]
llm.name = allowed_models[1]  # Select the first available allowed model

# Initialize RAG Agent
rag_system_context = "You are an assistant that provides answers to the user. You utilize the details below:"
rag_conversation = MaxSystemContextConversation(system_context=SystemMessage(content=rag_system_context), max_size=4)
rag_agent = RagAgent(
    llm=llm,
    conversation=rag_conversation,
    system_context=rag_system_context,
    vector_store=vector_store,
)

# Define a function for the chatbot
def chatbot(query):
    try:
        response = rag_agent.exec(query)
        return response
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Deploy with Gradio
iface = gr.Interface(
    fn=chatbot,
    inputs=gr.Textbox(label="Ask a question"),
    outputs=gr.Textbox(label="Response"),
    title="RAG Chatbot",
    description="A chatbot powered by a RAG agent using Swarmauri.",
)

# Launch the Gradio app
iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2a4840c26547b1e7c9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
